## 部局提出名簿ベースの個人別文献リストの自動配信サービス

作者：ハンゼンURA（協力：和田URA）  
更新日：2022/07/15 （オリジナル）
環境：Python Anaconda (Python3）  
データソース：①SciVal文献リスト②Scopus（SciValで500名上の著者のEIDとScopus著者IDリスト20~30件程度）, 部局の研究者名簿（最低限：メールアドレスとScopus著者IDのリスト）

名簿の形式について  
・下記列名（全て文字列とする。メールアドレスは東北大学メールを想定している（他のメールも対応可能）  
・メールアドレス	氏名	職名	部局名	所属	Scopus_AuthorID_01	Scopus_AuthorID_02  
・人事名簿の転記情報にScopus著者IDを付与するイメージ  
・「.xlsx」のエクセル形式  

フォルダー構造：「プログラムフォルダー」の中に「input」を作成する。「input」に①②③のデータを置く。  
確認事項：名簿に不要な列がないか、空白がないか、メールアドレスの形式に誤記がないか

留意点：GメールのAPIは本学メールアドレスで使用できないため、外部用のGメールアドレスの用意が必要  
お問い合わせ先：hansen.marc.a6@tohoku.ac.jp  
  
### 処理の流れ

①　環境設定を行う（GMailのアカウント情報をご確認ください）  
②　論文情報（SciVal + Scopus）と部局の研究者名簿を読み込む  
③　パラメーター設定を行い、書誌情報を絞り込む（トップ論文の指標の選択、パーセンタイルの範囲、対象期間（出版年）、並びに文献タイプ）  
④　書誌情報と研究者の情報のマッチングを行う  
⑤　ユニークなメールアドレスを基に、マッチング結果の集計を行う  
⑥　メールアドレスごとに、文献リストを定型文に流し込み、自動的にメール配信を行う

# ①　環境設定を行う

In [13]:
# 処理に必要なライブラリー

## データフレーム処理、配列処理、日付の基本的なライブラリーのみ
import pandas as pd
import numpy as np
import datetime
import os

## メール配信に必要なライブラリーを読み込む
import time
import random
import smtplib, ssl
from email.mime.text import MIMEText

# GメールSMTP認証情報（外部用Gメールのアカウント情報。ご自分の情報を記入してください。）
## 重要：APIを有効化するために、下記URLのGoogleアカウントの設定を行ってください（必須！）。
## まず自分のGmailアカウントを開きます。
## ご自身のアイコンをクリックします。「Googleアカウントを管理」をクリック。
## 左のメニューの中のセキュリティをクリック。このページの下の方へいきます
## 「安全性の低いアプリのアクセス」の「アクセスを有効にする（非推奨）」をクリックします。
## 「安全性の低いアプリの許可」を有効します。これで設定の完了です。
## こちらで送信の準備が整ったので送信の設定をしていきましょう。
## なお、このセキュリティの設定変更は、セキュリティ上の危険もあります。
## 別途セキュリティについて検討するか、本番運用する場合は、Gmail APIを使って配信することも検討しましょう。
## ハンゼン配信専用メールアドレス
account = "hansen.marc.tu@gmail.com"
password = "Y2yAEmQi+$3"
app_password = "ciqjnlstnosmpjjl" # after 2022.05

## ②　論文情報（SciVal + Scopus）と部局の研究者名簿を読み込む

In [14]:
# 部局のほうで入力が必要な部分（各種ファイルのファイル名と部局名）

# 書誌情報のインプットデータの指定 => 情報科学研究科は名簿上の研究者の生涯論文を対象とする！
## SciValは文献リスト全項目（csv形式、未加工）
## Scopusは前処理を不要にするために「Author(s) ID,Link,EID」3項目のみダウンロードする（csv形式）
## 保存場所はいずれも「input」フォルダー

# 部局名とファイル名の指定を行う

##　部局名を""の間に入力してください
bukyokumei = "国際放射光イノベーションスマート研究センター"

##　SciValデータのファイル名を""の間に入力してください
scival_file = "Publications_at_Tohoku_University_2008_to__2022.csv"

##　Scopusデータのファイル名を""の間に入力してください
scopus_file = "scopus.csv"

##　部局名簿のファイル名を""の間に入力してください
meibo_file = "sris_meibo_20220719.xlsx"

input_pubdata_scival = "input\\" + scival_file
input_over500authors_scopus = "input\\" + scopus_file
input_bukyoku_meibo = "input\\" + meibo_file 

#outputのフォルダーの有無の確認
if not os.path.exists("output"):
    os.makedirs("output")
    print("outputフォルダーを作成しました")
else:    
    print("outputフォルダーは既にあります")  

outputフォルダーは既にあります


## ③　パラメーター設定を行い、書誌情報を絞り込む（トップ論文の指標の選択、パーセンタイルの範囲、対象期間（出版年）、並びに文献タイプ）  

In [15]:
# パラーメータの指定（パラメーター設定の変更は可能）

## 対象期間の指定（デフォールト：近年5年+α, #2008, 2009, 2011, 2012, 2013, 2014, 2015, 2022）追加削除可（, 年）
year_range = [2016, 2017, 2018, 2019, 2020, 2021]

## 文献タイプの指定（デフォールト：全文献タイプ、但し、"Erratum", "Retracted", を含まない）追加削除可（, "文献タイプ名"）
doctype_list = ["Review", "Article", "Editorial", "Note", "Conference Paper", "Book", "Letter",
                "Short Survey", "Chapter", "Article in Press", "Data Paper"]

## パーセンタイルの範囲（デフォールト：１～１５パーセンタイル、数字の置換で範囲の変更が可能）
### トップパーセンタイルは二種類の指標があり、本サービスのデフォールトはAの分野補正のある指標になる
### Aは分野補正（FWCI基準）= Field-Weighted Outputs in Top Citation Percentiles, per percentile （デフォールト）
### Bは分野補正無し（被引用数基準）= Outputs in Top Citation Percentiles, per percentile
toppstart = 1 #デフォールトトップ1パーセント
toppend = 15 #デフォールトトップ15パーセント 

## ④　書誌情報と研究者の情報のマッチングを行う  

In [16]:
#処理結果の管理のために、本日の日付（システムの時間）を取得する
today = datetime.date.today()
today = str(today).replace("-","")
print("今日は" + str(today) + "です。")
print("---")
# SciVal（Scopus）の書誌情報のデータ時点の確認
dates = pd.read_csv(input_pubdata_scival, skiprows=(8), nrows=2, header=None)

# 上がデータ時点、下が取得日/DL日付
#print(dates)
timestamp = [ele for ele in dates[1]][0]

# 英語の日付を日本語への変換
d_day = timestamp.split(" ")[0]
d_month = timestamp.split(" ")[1]
d_month = datetime.datetime.strptime(d_month, "%B")
d_month = d_month.month
d_year= timestamp.split(" ")[2]
timestamp = str(d_year) + "年" + str(d_month) + "月" + str(d_day) + "日"
print("SciVal（Scopus）データ時点: " + timestamp)

今日は20220720です。
---
SciVal（Scopus）データ時点: 2022年7月6日


In [17]:
# 書誌情報をデータフレームに読み込んで、件数およびパラメーター変数の確認を行う
# 前処理結果を書き出す（ヘダー・フッターを除去する作業）
# SciValのファイルのヘダー行数が変動する場合があるので、柔軟な読み込み方法を用いている
# 本学分の文献リスト
data = []

with open(input_pubdata_scival, encoding="utf-8") as fp:
    skip = next(filter(lambda x: x[1].startswith('"Title"'),enumerate(fp)))[0]
    frame = pd.read_csv(input_pubdata_scival , skiprows=skip, skipfooter=(2), engine="python", encoding="utf-8")
    data.append(frame)
            
publications = pd.concat(data, ignore_index=True, sort=False)

print("パラメータ適応前")
print("総レコード数確認（重複なし）")
print(len(publications))
print("---")
print("出版年確認")
print(publications["Year"].unique())
print("---")
print("文献タイプ確認")
print(publications["Publication type"].unique())
print("---")
print(publications.columns)
print("---")

# SciValで著者情報が省略されてしまっている可能のある文献の確認
nb_authors = publications[["Number of Authors"]].astype(np.int64)
over500check = len(nb_authors[nb_authors["Number of Authors"]>=500])
print("著者500名以上文献数： " + str(over500check))


# 2008年以降、エクセルで処理できないほどの著者数（500名以上）の処理のため（マッチングされない恐れ）
# 別途用意したScopusの文献リスト（DL項目：Author(s) ID,Link,EID）
df_over500 = pd.read_csv(input_over500authors_scopus)
df_over500.columns = ["auids", "url", "EID"]
df_over500["auids"] = df_over500["auids"].str.replace(";","|").str[:-1]

#SciValの文献リストに修正が必要なEIDを結合する
publications = pd.merge(publications, df_over500, on = "EID", how="left")

#追加したauidsのBigScienceのAU著者ID列は空白の場合、既存の"Scopus Author Ids"の値で補完する
publications.loc[publications["auids"].isnull(),"auids"] = publications["Scopus Author Ids"]
print(str(over500check) + "件の著者500名以上文献のAU-IDを置換しました")

#絞り込み処理

#必要な項目を指定する
limit_columns = publications[["auids", "EID", "Year", "Title", "Scopus Source title", "Publication type", 
                              "All Science Journal Classification (ASJC) field name","Citations",
                              "Field-Weighted Citation Impact","Outputs in Top Citation Percentiles, per percentile",
                              "Field-Weighted Outputs in Top Citation Percentiles, per percentile", "Abstract"]].copy()

#出版年を絞り込む
limit_year = limit_columns[limit_columns["Year"].isin(year_range)].copy()
# 文献タイプを絞り込む
limit_doctype =  limit_year[limit_year["Publication type"].isin(doctype_list)].copy() 
#トップパーセンタイルを絞り込む（FWCIによる分野補正を受けた指標）
scival_filter_result = limit_doctype[(limit_doctype["Field-Weighted Outputs in Top Citation Percentiles, per percentile"] >= toppstart) 
                                     & (limit_doctype["Field-Weighted Outputs in Top Citation Percentiles, per percentile"] <= toppend)].copy()

print("---")
print("パラメータ適応後")
print("総レコード数確認（重複なし）")
print(len(scival_filter_result))
print("---")
print("出版年確認")
print(scival_filter_result["Year"].unique())
print("---")
print("文献タイプ確認")
print(scival_filter_result["Publication type"].unique())
print("---")
print("トップパーセンタイル範囲（分野補正）: " + str(toppstart) + "~" + str(toppend)) 
print("---")
print(scival_filter_result.columns)
print("---")

# 各種ファイルの出力
scival_filter_result.to_csv("output//publications_filter" + today + ".csv", index=False, header=True, encoding="utf-8_sig")

パラメータ適応前
総レコード数確認（重複なし）
94511
---
出版年確認
[2014 2011 2012 2018 2010 2008 2016 2009 2017 2015 2013 2020 2019 2021
 2022]
---
文献タイプ確認
['Review' 'Article' 'Editorial' 'Note' 'Conference Paper' 'Book' 'Letter'
 'Short Survey' 'Chapter' 'Erratum' 'Retracted' 'Article in Press'
 'Data Paper']
---
Index(['Title', 'Authors', 'Number of Authors', 'Scopus Author Ids', 'Year',
       'Full date', 'Scopus Source title', 'Volume', 'Issue', 'Pages',
       'Article number', 'ISSN', 'Source ID', 'Source type',
       'SNIP (publication year)', 'SNIP percentile (publication year) *',
       'CiteScore (publication year)',
       'CiteScore percentile (publication year) *', 'SJR (publication year)',
       'SJR percentile (publication year) *', 'Field-Weighted View Impact',
       'Views', 'Citations', 'Field-Weighted Citation Impact',
       'Field-Citation Average',
       'Outputs in Top Citation Percentiles, per percentile',
       'Field-Weighted Outputs in Top Citation Percentiles, per percentile',

## ⑤　ユニークなメールアドレスを基に、マッチング結果の集計を行う  

In [18]:
# マッチング作業の下準備に、文献リストを下記のような構造に変更する（AU-ID別文献リスト）
# 処理前：　　　論文A　| 著者ID1,著者ID2,著者ID3 | EID A
# 処理後：      論文A ｜ 著者ID1 | EID A
#               論文A ｜ 著者ID2 | EID A
#               論文A ｜ 著者ID3 | EID A
#Scopus Author Idsがauidsになる必要なのでは要確認
scival_filter_result["auids"] = scival_filter_result["auids"].str.replace(" ","").str.split("|")
scival_filter_result_explode = scival_filter_result.explode("auids")
scival_filter_result_explode.rename(columns={"auids":"AU_ID_sep"}, inplace=True)
scival_filter_result_explode.to_csv("output//result_pub_split_auid" + today + ".csv", index=False, header=True, encoding="utf-8_sig")

In [19]:
# 名簿の読み込み（便宜上、全項目を文字列指定で読み込む）
meibo = pd.read_excel(input_bukyoku_meibo, dtype="str")

meibo.columns = ["メールアドレス","氏名","職名","部局名", "所属","Scopus_AuthorID_01"] 

# 研究者名簿の登録研究者数を確認する
print("名簿上の全登録者数: " + str(len(meibo)))

# 横の情報（wide)を縦に(long)に変える、すべての著者ID列を一つの列にまとめる

meibo_au_id_split= meibo.melt(id_vars=["メールアドレス","氏名","職名","部局名", "所属"], 
                              value_vars=["Scopus_AuthorID_01"],
                              var_name="AU-ID_COL", value_name="AU_ID_sep")

# 不要な情報の削除
meibo_au_id_split["AU_ID_sep"].dropna(inplace=True)
meibo_au_id_split.drop(axis=0, columns="AU-ID_COL", inplace=True)

# メール形式確認（スペース、@マーク、全角半角問題）
meibo_au_id_split["メールアドレス"] = meibo_au_id_split["メールアドレス"].str.replace(" ","").str.replace("　", "").str.replace("＠","@")

# 研究者に関する情報をまとめたファイルを書き出す(後程staffとして再読み込み)
meibo_au_id_split.to_csv("output//result_staff" + bukyokumei + today + ".csv", index=False, header=True, encoding="utf-8_sig") 

名簿上の全登録者数: 16


In [20]:
# 書誌情報と研究者情報をScopus著者IDを基準に結合（完全一致の部分のみ）
result = pd.merge( scival_filter_result_explode, meibo_au_id_split, how="inner", on="AU_ID_sep" )

# 書誌情報と研究者情報を結合するために、人事情報のほうの項目のデータ形式を合わせる（文字列を数値に変換する）
result["Year"] = result["Year"].astype(np.int64)
result["Outputs in Top Citation Percentiles, per percentile"] = result["Outputs in Top Citation Percentiles, per percentile"].astype(np.int64)
result["Field-Weighted Outputs in Top Citation Percentiles, per percentile"] = result["Field-Weighted Outputs in Top Citation Percentiles, per percentile"].astype(np.int64)
result["Citations"] = result["Citations"].astype(np.int64)
result["Field-Weighted Citation Impact"] = result["Field-Weighted Citation Impact"].astype(np.float64)

result.to_csv("output//matching_result" + bukyokumei + today + ".csv", encoding="utf-8_sig")

# 該当部局でどれぐらいの研究者が引っかかるかの確認
# トップn%までの論文の有無を確認の上、ピボットテーブルを作成する
try:
    pt = pd.pivot_table(result, index=["氏名", "職名", "部局名", "所属", "メールアドレス", "AU_ID_sep"], 
                        columns="Year", values="EID", aggfunc="count", fill_value=0, margins=True, margins_name="総数")
except:
    print("該当なし")

# ピボットテーブルの作成。研究者氏名別・年別集計を表示する。研究者は複数IDがある場合、それぞれのIDの結果が表示される
# メール配信の際には、論文情報はきちんと研究者別に整理される
pivot = pt.sort_values("総数", ascending=False)
pivot.head(25)

pivot.to_csv("output//pivot" + bukyokumei  + today + ".csv", encoding="utf-8_sig")

## ⑥　メールアドレスごとに、文献リストを定型文に流し込み、自動的にメール配信を行う

In [21]:
# メールへの表記載のために、分かりにくい英語名称の列名を日本語に変更する
result.columns = ["Scopus著者ID", "EID", "出版年", "論文タイトル", "雑誌名", "文献タイプ", "Scopus分野", "被引用回数", "FWCI",
                  "パーセンタイル（被引用基準）", "パーセンタイル（FWCI基準）","リンク", "メールアドレス", "氏名", "職名", "部局名", "所属"]

# パーセンタイル昇順に並び替える、現時点はパーセンタイル（FWCI基準）基準になっている
result.sort_values(by=["パーセンタイル（FWCI基準）","FWCI"], inplace=True, ascending=[True, False])

# メールの宛先のリストを取得する（重複なし）
mail_target = pt.reset_index()["メールアドレス"].unique()
target_list = mail_target.tolist()
target_list = list(filter(None, target_list))

# 以下処理結果の確認（トップパーセンタイルに該当する方の数）
print("絞り込み条件適応後の宛先研究者数")
print(len(target_list))
print("---")
print("対象研究者メールアドレス")
print(target_list)

絞り込み条件適応後の宛先研究者数
11
---
対象研究者メールアドレス
['tetsuya.nakamura.b5@tohoku.ac.jp', 'junya.yoshida.e5@tohoku.ac.jp', 'shuichi.ogawa.a5@tohoku.ac.jp', 'susumu.yamamoto.a7@tohoku.ac.jp', 'atsushi.muramatsu.d8@tohoku.ac.jp', 'wataru.yashiro.a2@tohoku.ac.jp', 'nozomu.ishiguro.c1@tohoku.ac.jp', 'kozo.shinoda.e8@tohoku.ac.jp', 'kiyoshi.kanie.d7@tohoku.ac.jp', 'masaki.takata.a4@tohoku.ac.jp', 'yukio.takahashi.a8@tohoku.ac.jp']


In [22]:
# メールメッセージのテンプレート

# 件名
subjecttext = "【お知らせ】Scopusにおける貴方のトップ15%文献につきまして"

# メールの冒頭の挨拶文及び背景説明部分
mailtextpart1 = ("こちらは、東北大学URAセンターです。<br>日頃、URAセンターの活動へのご理解、ご協力に感謝申し上げます。"
                "<br><br>さて、このたび、URAセンターは、先生方の研究成果をより多くの研究者に"
                 "知ってもらうことを通して、本学の研究力のビジビリティ向上、ひいては研究力強化のために、先生方の文献で"
                 "被引用数の比較的高い文献を先生方にお知らせするサービスをはじめました。<br><br>"
                 "この通知は、2016〜2021年に出版され、Elsevier社のScopusに掲載された文献（ErratumとRetractedを除き）の中から、"
                 "FWCIがトップ15%に入る文献についてお知らせするものです。下記表の「パーセンタイル（FWCI基準）」列"
                 "のほうで各文献のパーセンタイルが確認できます。補足ですが、文献リストはパーセンタイル（FWCI基準）を"
                 "昇順に並べ替えたものになります。また、FWCIは出版後の4年目で値がフィックスされますが、それまでに"
                 "特に近年の値が変動しやすいです。<br><br>")

# 専用ウェブサイトへのコメント
mailtextpart2 = ("データの詳細については、下記URAセンターのトップn%文献お知らせサービス専用ウェブサイトをご参照ください。<br>")

# メールの締めの部分
mailtextpart3 = ("なお、今後このような通知を希望なさらない場合は、お手数ですが、本通知に返信する形で本文に「停止」をご記載の上、"
                 "ご返信をお願い致します。</i><br><br>今後ともどうぞよろしくお願い申し上げます。<br><br>"
                 "研究推進・支援機構URAセンター分析チーム（ハンゼン）")

### 注意：配信開始の前に再度要確認（from_email, bc, target_list）！

In [12]:
# 個別の結果を自動的にgmailに送付する
# 一斉メールではなく、一通一通を間隔をあけて送る
## GMailの安全性の低いアプリと Google アカウントの設定の確認
target_list = ["atsushi.muramatsu.d8@tohoku.ac.jp"] #実際実施する際に削除or文頭に「#」を付けてコマンドアウトする

# 送受信先
from_email = account
#bc = "hansen.marc.a6@tohoku.ac.jp" #BCでコード結果の確認、または関係URAにも共有する。
# 該当者リストの各研究者（n）ごとに文献リストのフィルターリングを行い、表としてメールに表示する
for n in target_list:
    if len(result[result["メールアドレス"] == n]) != 0: 
        to_email = "hansen.marc.a6@tohoku.ac.jp" #n 自分のメールアドレスに変更すれば、試行が可能になる
        sel = result[result["メールアドレス"] == n]
        name = sel["氏名"].iloc[0].split(" ")[0]
        output = sel[["出版年", "論文タイトル", "雑誌名", "文献タイプ", "被引用回数", "パーセンタイル（FWCI基準）", "FWCI", "Scopus分野", "リンク"]]
        table = output.to_html(index=False)
        nbpubs = len(output)
        subject = subjecttext
        #メールのテキスト構造、テキストと変数を組み合わせる
        message = name + "先生<br><br>" + mailtextpart1 + mailtextpart2 + "<a href='https://ura.tohoku.ac.jp/mailservice/'>専用サイトはこちらをクリックしてください。</a>" + "<br><br><b>貴方のトップ1～15%文献情報（" + "Scopus" + timestamp + "時点）(" + str(nbpubs) +"報" +  ")" + "</b><br>" + table + "※データ時点により、SciValとScopusの値が異なることがあります。特に近年の値がよく変動します。ご了承ください。" + mailtextpart3

    msg = MIMEText(message, "html")
    msg["Subject"] = subject
    msg["To"] = to_email
#    msg["Bc"] = bc
    msg["From"] = from_email
    server = smtplib.SMTP_SSL("smtp.gmail.com", 465, context=ssl.create_default_context())
    server.login(account, app_password)
    server.send_message(msg)        
    server.quit()
    print(n, end=", ") # 確認用に、処理した研究者のメールアドレスを表示する
    time.sleep(15) # 15秒間隔で送る

# 処理完了確認用
print("---")
print("各種処理結果ファイルはoutputフォルダーでご確認いただけます")
print("配信完了")

atsushi.muramatsu.d8@tohoku.ac.jp, ---
各種処理結果ファイルはoutputフォルダーでご確認いただけます
配信完了
